In [1]:
import pandas as pd

from tqdm import tqdm
tqdm.pandas()

import gensim.downloader
import numpy as np

from pathlib import Path

import glob
import re

# Создание конечного датафрейма

In [2]:
# columns_data_path = "./collecting/columns_data.csv"
# if not Path(columns_data_path).exists():
#     raise Exception("columns_data.csv doesnt exist. Run `make; ./collect_columns_data` from this dir.")

# df_data = pd.read_csv(f"columns_data.csv", sep="⇧", engine="python", quotechar='"', on_bad_lines="warn")

In [3]:
import pandas as pd
import glob

df_list = []

number_of_chunks = len(glob.glob("../collecting/columns_data/data*.csv"))
for i in range(number_of_chunks):
    df_list.append(
        pd.read_csv(f"../collecting/columns_data/data_{i}.csv", sep="⇧", engine="python", quotechar='"', on_bad_lines="warn")
    )

df_data = pd.concat(df_list, axis=0)

Skipping line 669: '⇧' expected after '"'
Skipping line 3367: '⇧' expected after '"'
Skipping line 4578: '⇧' expected after '"'
Skipping line 6906: '⇧' expected after '"'
Skipping line 6907: '⇧' expected after '"'
Skipping line 10001: '⇧' expected after '"'
Skipping line 10696: '⇧' expected after '"'
Skipping line 11865: '⇧' expected after '"'
Skipping line 12034: '⇧' expected after '"'
Skipping line 15181: '⇧' expected after '"'
Skipping line 16006: '⇧' expected after '"'
Skipping line 16834: '⇧' expected after '"'
Skipping line 17469: '⇧' expected after '"'
Skipping line 17769: '⇧' expected after '"'
Skipping line 18899: '⇧' expected after '"'
Skipping line 20430: '⇧' expected after '"'
Skipping line 27577: '⇧' expected after '"'
Skipping line 27801: '⇧' expected after '"'
Skipping line 27821: '⇧' expected after '"'
Skipping line 28179: '⇧' expected after '"'
Skipping line 28647: '⇧' expected after '"'
Skipping line 30059: '⇧' expected after '"'
Skipping line 30720: '⇧' expected afte

In [4]:
df_data_filter = df_data[~df_data["column_data"].isna()]
df_data_filter

,table_id,column_id,column_data
0,dataset/Dataset/data/1000006/table_0,0,Театр им. Евгения Вахтангова (Москва) Театр им...
1,dataset/Dataset/data/1000020/table_0,0,1963 1965 1968 1969 1970 1970 1971 1971 1971 1...
2,dataset/Dataset/data/1000050/table_0,0,"1984, 1987, 1991 1984, 1985, 1987 1991"
3,dataset/Dataset/data/1000078/table_1,2,Оранжевая корова
4,dataset/Dataset/data/1000190/table_0,0,2003
...,...,...,...
125170,dataset/Dataset/data/999585/table_2,1,«Дай мне силу» «Настройся на меня» «В комнате ...
125171,dataset/Dataset/data/999639/table_2,1,Победитель Второе место Третье место Финиширов...
125172,dataset/Dataset/data/999851/table_0,1,14 августа 1903 30 августа 1900 11 мая 1900 22...
125173,dataset/Dataset/data/999913/table_1,0,28-4-1 28-3-1 28-2-1 28-2 28-1 27-1 26-1 25-1 ...


## Очистка мета-информации

- Очищена мета-информация .mv-parser...
- Очищены ссылки типа ***some-text[1]***.

### .mw-parser

In [5]:
mv_parser = df_data_filter["column_data"].str.contains(".mw-parser")
df_data_filter[mv_parser]

,table_id,column_id,column_data
20,dataset/Dataset/data/1000933/table_1,5,22 февраля 1993.mw-parser-output .ts-comment-c...
667,dataset/Dataset/data/104134/table_0,3,Текст Текст Текст Текст Текст Текст https://ru...
3387,dataset/Dataset/data/1058096/table_0,0,1985 1986 1989 1989 1989 1990 1990 1991 1991 1...
4204,dataset/Dataset/data/1106461/table_0,0,19.06.2001 03.08.2003 27.01.2005 14.10.2005 24...
4553,dataset/Dataset/data/1136426/table_1,1,A Б A Б A Б В A Б В A Б В A Б В A Б В A Б В А ...
...,...,...,...
120428,dataset/Dataset/data/8779389/table_3,5,(.mw-parser-output .ts-comment-commentedText{b...
122243,dataset/Dataset/data/8894249/table_0,3,"116 238 530 2.06,0 4.19,1 16.38,4 34.11,6 .mw-..."
123461,dataset/Dataset/data/8977689/table_3,4,.mw-parser-output .ts-comment-commentedText{bo...
123548,dataset/Dataset/data/8982369/table_0,4,.mw-parser-output .ts-comment-commentedText{bo...


In [6]:
df_data_filter.loc[:, "column_data"] = df_data_filter["column_data"].progress_apply(
    lambda x: re.sub(r"\.mw-parser-output \.ts.+{.+}", " ", x)
)
df_data_filter["column_data"].str.contains("mw-parser").sum()

100%|████████████████████████████| 1448585/1448585 [00:01<00:00, 1103682.01it/s]


0

In [7]:
# def clean_mv_parser(col_name: str) -> str:
#     if col_name.strip().startswith(".mw-parser"):
#         return col_name.split("}")[-1]
#     return col_name

# df_data_filter.loc[:, "column_data"] = df_data_filter["column_data"].progress_apply(lambda x: clean_mv_parser(str(x)))
# df_data_filter

In [8]:
df_data_filter[mv_parser]

,table_id,column_id,column_data
20,dataset/Dataset/data/1000933/table_1,5,22 февраля 1993 28 ноября 1997[13]1 декабря 19...
667,dataset/Dataset/data/104134/table_0,3,Текст Текст Текст Текст Текст Текст https://ru...
3387,dataset/Dataset/data/1058096/table_0,0,1985 1986 1989 1989 1989 1990 1990 1991 1991 1...
4204,dataset/Dataset/data/1106461/table_0,0,19.06.2001 03.08.2003 27.01.2005 14.10.2005 24...
4553,dataset/Dataset/data/1136426/table_1,1,A Б A Б A Б В A Б В A Б В A Б В A Б В A Б В А ...
...,...,...,...
120428,dataset/Dataset/data/8779389/table_3,5,( с/в) ЕР ЕР ЕР ЕР ГП ЕР ЕР ЕР ЕР ЕР ЕР ЕР ЕР ...
122243,dataset/Dataset/data/8894249/table_0,3,"116 238 530 2.06,0 4.19,1 16.38,4 34.11,6 NR*..."
123461,dataset/Dataset/data/8977689/table_3,4,Q Q Q Q Q Q q q PB NR
123548,dataset/Dataset/data/8982369/table_0,4,DNF DNF 111 120 137


### Ссылки википедии

In [9]:
import re

df_data_filter.loc[:, "column_data"] = df_data_filter["column_data"].progress_apply(lambda x: re.sub(r"\[[0-9]*\]", "", x))
df_data_filter

100%|████████████████████████████| 1448585/1448585 [00:01<00:00, 1061474.52it/s]


,table_id,column_id,column_data
0,dataset/Dataset/data/1000006/table_0,0,Театр им. Евгения Вахтангова (Москва) Театр им...
1,dataset/Dataset/data/1000020/table_0,0,1963 1965 1968 1969 1970 1970 1971 1971 1971 1...
2,dataset/Dataset/data/1000050/table_0,0,"1984, 1987, 1991 1984, 1985, 1987 1991"
3,dataset/Dataset/data/1000078/table_1,2,Оранжевая корова
4,dataset/Dataset/data/1000190/table_0,0,2003
...,...,...,...
125170,dataset/Dataset/data/999585/table_2,1,«Дай мне силу» «Настройся на меня» «В комнате ...
125171,dataset/Dataset/data/999639/table_2,1,Победитель Второе место Третье место Финиширов...
125172,dataset/Dataset/data/999851/table_0,1,14 августа 1903 30 августа 1900 11 мая 1900 22...
125173,dataset/Dataset/data/999913/table_1,0,28-4-1 28-3-1 28-2-1 28-2 28-1 27-1 26-1 25-1 ...


In [10]:
empty_column_data_mask = df_data_filter["column_data"].apply(lambda x: str(x).replace(";", "").strip()) == ""

In [11]:
df_data_filter = df_data_filter[~empty_column_data_mask]

## JOIN заголовков с содержимым столбца

создание композитного ключа и merge(inner) двух датафреймов

### ColumnData dataframe

In [12]:
df_data_filter.loc[:, "composite_key"] = df_data_filter["table_id"] + " / " + df_data_filter["column_id"].astype(str)
df_data_filter

/tmp/ipykernel_4018/3755119166.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_filter.loc[:, "composite_key"] = df_data_filter["table_id"] + " / " + df_data_filter["column_id"].astype(str)


,table_id,column_id,column_data,composite_key
0,dataset/Dataset/data/1000006/table_0,0,Театр им. Евгения Вахтангова (Москва) Театр им...,dataset/Dataset/data/1000006/table_0 / 0
1,dataset/Dataset/data/1000020/table_0,0,1963 1965 1968 1969 1970 1970 1971 1971 1971 1...,dataset/Dataset/data/1000020/table_0 / 0
2,dataset/Dataset/data/1000050/table_0,0,"1984, 1987, 1991 1984, 1985, 1987 1991",dataset/Dataset/data/1000050/table_0 / 0
3,dataset/Dataset/data/1000078/table_1,2,Оранжевая корова,dataset/Dataset/data/1000078/table_1 / 2
4,dataset/Dataset/data/1000190/table_0,0,2003,dataset/Dataset/data/1000190/table_0 / 0
...,...,...,...,...
125170,dataset/Dataset/data/999585/table_2,1,«Дай мне силу» «Настройся на меня» «В комнате ...,dataset/Dataset/data/999585/table_2 / 1
125171,dataset/Dataset/data/999639/table_2,1,Победитель Второе место Третье место Финиширов...,dataset/Dataset/data/999639/table_2 / 1
125172,dataset/Dataset/data/999851/table_0,1,14 августа 1903 30 августа 1900 11 мая 1900 22...,dataset/Dataset/data/999851/table_0 / 1
125173,dataset/Dataset/data/999913/table_1,0,28-4-1 28-3-1 28-2-1 28-2 28-1 27-1 26-1 25-1 ...,dataset/Dataset/data/999913/table_1 / 0


### Threshold dataframe

In [13]:
threshold = 0.78
df_threshold = pd.read_csv(f"../labelling/labelled/threshold_{threshold}.csv", sep=";")
df_threshold

,table_id,column_id,column_name_lemma,label,similarity
0,dataset/Dataset/data/1000006/table_0,0,театр,театр,1.000000
1,dataset/Dataset/data/1000006/table_0,1,спектакль,пьеса,0.789814
2,dataset/Dataset/data/1000006/table_0,4,примечание,примечание,1.000000
3,dataset/Dataset/data/1000006/table_1,2,название,название,1.000000
4,dataset/Dataset/data/1000006/table_1,0,год,год,1.000000
...,...,...,...,...,...
1504998,dataset/Dataset/data/999987/table_0,6,результат,результат,1.000000
1504999,dataset/Dataset/data/999987/table_0,2,турнир,турнир,1.000000
1505000,dataset/Dataset/data/999987/table_0,0,год,год,1.000000
1505001,dataset/Dataset/data/999987/table_0,7,место,место,1.000000


In [14]:
df_threshold.drop(["column_name_lemma", "similarity"], axis=1, inplace=True)
df_threshold

,table_id,column_id,label
0,dataset/Dataset/data/1000006/table_0,0,театр
1,dataset/Dataset/data/1000006/table_0,1,пьеса
2,dataset/Dataset/data/1000006/table_0,4,примечание
3,dataset/Dataset/data/1000006/table_1,2,название
4,dataset/Dataset/data/1000006/table_1,0,год
...,...,...,...
1504998,dataset/Dataset/data/999987/table_0,6,результат
1504999,dataset/Dataset/data/999987/table_0,2,турнир
1505000,dataset/Dataset/data/999987/table_0,0,год
1505001,dataset/Dataset/data/999987/table_0,7,место


In [15]:
df_threshold.loc[:, "composite_key"] = df_threshold["table_id"] + " / " + df_threshold["column_id"].astype(str)
df_threshold

,table_id,column_id,label,composite_key
0,dataset/Dataset/data/1000006/table_0,0,театр,dataset/Dataset/data/1000006/table_0 / 0
1,dataset/Dataset/data/1000006/table_0,1,пьеса,dataset/Dataset/data/1000006/table_0 / 1
2,dataset/Dataset/data/1000006/table_0,4,примечание,dataset/Dataset/data/1000006/table_0 / 4
3,dataset/Dataset/data/1000006/table_1,2,название,dataset/Dataset/data/1000006/table_1 / 2
4,dataset/Dataset/data/1000006/table_1,0,год,dataset/Dataset/data/1000006/table_1 / 0
...,...,...,...,...
1504998,dataset/Dataset/data/999987/table_0,6,результат,dataset/Dataset/data/999987/table_0 / 6
1504999,dataset/Dataset/data/999987/table_0,2,турнир,dataset/Dataset/data/999987/table_0 / 2
1505000,dataset/Dataset/data/999987/table_0,0,год,dataset/Dataset/data/999987/table_0 / 0
1505001,dataset/Dataset/data/999987/table_0,7,место,dataset/Dataset/data/999987/table_0 / 7


### Join dataframes

In [16]:
df_joined = df_threshold.merge(df_data_filter[["composite_key", "column_data"]], how="inner", on="composite_key")
df_joined.drop(["composite_key"], axis=1, inplace=True)
df_joined

,table_id,column_id,label,column_data
0,dataset/Dataset/data/1000006/table_0,0,театр,Театр им. Евгения Вахтангова (Москва) Театр им...
1,dataset/Dataset/data/1000006/table_0,1,пьеса,Дневник Анны ФранкФрэнсис Гудрич и Альберт Хак...
2,dataset/Dataset/data/1000006/table_0,4,примечание,видеофрагмент видеофрагмент телеспектакль виде...
3,dataset/Dataset/data/1000006/table_1,2,название,"Кулагин и партнёры Люба, дети и завод… Врачебн..."
4,dataset/Dataset/data/1000006/table_1,0,год,2004—2013 2006 2006 2006 2008 2007 2007 2008 2...
...,...,...,...,...
1441344,dataset/Dataset/data/999987/table_0,6,результат,7½ из 9 Нокаут-система 6½ из 9 8 из 11 6½ из 1...
1441345,dataset/Dataset/data/999987/table_0,2,турнир,71-й чемпионат Украины 73-й чемпионат Украины ...
1441346,dataset/Dataset/data/999987/table_0,0,год,2002 2004 2010 2012 2012 2013 2014 2016 2018
1441347,dataset/Dataset/data/999987/table_0,7,место,1 2 2 1 1 3 3 2 1


### Create label_id

In [17]:
labels = list(df_joined["label"].unique())
labels.sort()

sem_types = {v: i for i, v in enumerate(labels)}
sem_types

{'авиакомпания': 0,
 'автомобиль': 1,
 'адрес': 2,
 'актер': 3,
 'альбом': 4,
 'аннотация': 5,
 'арена': 6,
 'атлет': 7,
 'аэропорт': 8,
 'база': 9,
 'банк': 10,
 'боксер': 11,
 'борец': 12,
 'ввп': 13,
 'ведущий': 14,
 'вес': 15,
 'вещество': 16,
 'владелец': 17,
 'возраст': 18,
 'высота': 19,
 'гимнаст': 20,
 'глубина': 21,
 'год': 22,
 'гонка': 23,
 'город': 24,
 'группа': 25,
 'дата': 26,
 'двигатель': 27,
 'день': 28,
 'директор': 29,
 'документ': 30,
 'дорога': 31,
 'жанр': 32,
 'животное': 33,
 'журнал': 34,
 'закон': 35,
 'звук': 36,
 'здание': 37,
 'игра': 38,
 'идентификатор': 39,
 'идеология': 40,
 'издатель': 41,
 'изображение': 42,
 'инструмент': 43,
 'камера': 44,
 'канал': 45,
 'категория': 46,
 'класс': 47,
 'классификация': 48,
 'клуб': 49,
 'книга': 50,
 'код': 51,
 'количество': 52,
 'колледж': 53,
 'команда': 54,
 'компания': 55,
 'компонент': 56,
 'континент': 57,
 'конькобежец': 58,
 'корабль': 59,
 'кратер': 60,
 'лейбл': 61,
 'лес': 62,
 'лига': 63,
 'локомотив'

In [18]:
df_joined["label_id"] = df_joined["label"].progress_apply(lambda x: sem_types[x])
df_joined

100%|████████████████████████████| 1441349/1441349 [00:00<00:00, 2329413.08it/s]


,table_id,column_id,label,column_data,label_id
0,dataset/Dataset/data/1000006/table_0,0,театр,Театр им. Евгения Вахтангова (Москва) Театр им...,145
1,dataset/Dataset/data/1000006/table_0,1,пьеса,Дневник Анны ФранкФрэнсис Гудрич и Альберт Хак...,110
2,dataset/Dataset/data/1000006/table_0,4,примечание,видеофрагмент видеофрагмент телеспектакль виде...,101
3,dataset/Dataset/data/1000006/table_1,2,название,"Кулагин и партнёры Люба, дети и завод… Врачебн...",73
4,dataset/Dataset/data/1000006/table_1,0,год,2004—2013 2006 2006 2006 2008 2007 2007 2008 2...,22
...,...,...,...,...,...
1441344,dataset/Dataset/data/999987/table_0,6,результат,7½ из 9 Нокаут-система 6½ из 9 8 из 11 6½ из 1...,118
1441345,dataset/Dataset/data/999987/table_0,2,турнир,71-й чемпионат Украины 73-й чемпионат Украины ...,154
1441346,dataset/Dataset/data/999987/table_0,0,год,2002 2004 2010 2012 2012 2013 2014 2016 2018,22
1441347,dataset/Dataset/data/999987/table_0,7,место,1 2 2 1 1 3 3 2 1,66


### Reorder columns

In [19]:
df_joined = df_joined[["table_id", "column_id", "label_id", "label", "column_data"]]
df_joined

,table_id,column_id,label_id,label,column_data
0,dataset/Dataset/data/1000006/table_0,0,145,театр,Театр им. Евгения Вахтангова (Москва) Театр им...
1,dataset/Dataset/data/1000006/table_0,1,110,пьеса,Дневник Анны ФранкФрэнсис Гудрич и Альберт Хак...
2,dataset/Dataset/data/1000006/table_0,4,101,примечание,видеофрагмент видеофрагмент телеспектакль виде...
3,dataset/Dataset/data/1000006/table_1,2,73,название,"Кулагин и партнёры Люба, дети и завод… Врачебн..."
4,dataset/Dataset/data/1000006/table_1,0,22,год,2004—2013 2006 2006 2006 2008 2007 2007 2008 2...
...,...,...,...,...,...
1441344,dataset/Dataset/data/999987/table_0,6,118,результат,7½ из 9 Нокаут-система 6½ из 9 8 из 11 6½ из 1...
1441345,dataset/Dataset/data/999987/table_0,2,154,турнир,71-й чемпионат Украины 73-й чемпионат Украины ...
1441346,dataset/Dataset/data/999987/table_0,0,22,год,2002 2004 2010 2012 2012 2013 2014 2016 2018
1441347,dataset/Dataset/data/999987/table_0,7,66,место,1 2 2 1 1 3 3 2 1


In [20]:
df_joined[df_joined["table_id"] == "dataset/Dataset/data/1367439/table_0"]

,table_id,column_id,label_id,label,column_data
91674,dataset/Dataset/data/1367439/table_0,1,95,пол,неизвестен


In [21]:
column_data_na = df_joined["column_data"].isna().sum()
column_data_na
# df_joined = df_joined[~column_data_na]

0

In [22]:
df_joined

,table_id,column_id,label_id,label,column_data
0,dataset/Dataset/data/1000006/table_0,0,145,театр,Театр им. Евгения Вахтангова (Москва) Театр им...
1,dataset/Dataset/data/1000006/table_0,1,110,пьеса,Дневник Анны ФранкФрэнсис Гудрич и Альберт Хак...
2,dataset/Dataset/data/1000006/table_0,4,101,примечание,видеофрагмент видеофрагмент телеспектакль виде...
3,dataset/Dataset/data/1000006/table_1,2,73,название,"Кулагин и партнёры Люба, дети и завод… Врачебн..."
4,dataset/Dataset/data/1000006/table_1,0,22,год,2004—2013 2006 2006 2006 2008 2007 2007 2008 2...
...,...,...,...,...,...
1441344,dataset/Dataset/data/999987/table_0,6,118,результат,7½ из 9 Нокаут-система 6½ из 9 8 из 11 6½ из 1...
1441345,dataset/Dataset/data/999987/table_0,2,154,турнир,71-й чемпионат Украины 73-й чемпионат Украины ...
1441346,dataset/Dataset/data/999987/table_0,0,22,год,2002 2004 2010 2012 2012 2013 2014 2016 2018
1441347,dataset/Dataset/data/999987/table_0,7,66,место,1 2 2 1 1 3 3 2 1


### Remove stupid .mw-parser ._.

In [23]:
# df_joined.loc[:, "column_data"] = df_joined["column_data"].progress_apply(lambda x: re.sub(r"\.mw-parser-output \.ts.+{.+}", "", x))
df_joined["column_data"].str.contains("mw-parser").sum()

0

In [24]:
df_joined[df_joined["table_id"] == "dataset/Dataset/data/1012855/table_1"]

,table_id,column_id,label_id,label,column_data
2300,dataset/Dataset/data/1012855/table_1,4,106,продюсер,Роберт Латман Браун
2301,dataset/Dataset/data/1012855/table_1,7,51,код,0
2302,dataset/Dataset/data/1012855/table_1,1,73,название,«Вавилон-5: Сборы» «Babylon 5: The Gathering»
2303,dataset/Dataset/data/1012855/table_1,2,3,актер,Ричард Комптон
2304,dataset/Dataset/data/1012855/table_1,3,144,сценарист,Дж. Майкл Стражински


In [42]:
test_df = df_joined.sample(frac=0.05, random_state=13)
print(test_df["label"].unique().shape)
test_df

(170,)


,table_id,column_id,label_id,label,column_data
727059,dataset/Dataset/data/5340234/table_0,0,49,клуб,Алмалык Андижан Бунёдкор Бухара Динамо Коканд ...
245525,dataset/Dataset/data/2377813/table_3,0,22,год,2005 2005 2005 2005 2005 2006 2006 2007 2007 2...
1392140,dataset/Dataset/data/881108/table_3,2,72,награда,Лауреат XI Всесоюзного кинофестиваля в Ереване...
2405,dataset/Dataset/data/1013249/table_2,4,144,сценарист,Дж. Майкл Стражински Дж. Майкл Стражински Ларр...
1034684,dataset/Dataset/data/6867415/table_3,0,87,период,04.1945 — 07.1945 07.1945 — 1947 1947 — 1977 1...
...,...,...,...,...,...
344299,dataset/Dataset/data/3058088/table_0,2,73,название,Поступление Тусовщики Экстази Целуя Джессику С...
1252781,dataset/Dataset/data/7996420/table_5,2,73,название,«Сад смерти»(англ. «Garden of death») Семейств...
1387478,dataset/Dataset/data/8783283/table_2,3,66,место,6.0
244597,dataset/Dataset/data/2366647/table_12,1,25,группа,"«Heart of Glass» Blondie Джейми, Илиза, и Касс..."


In [44]:
train_df = df_joined.drop(test_df.index)
print(train_df["label"].unique().shape)
train_df

(170,)


,table_id,column_id,label_id,label,column_data
0,dataset/Dataset/data/1000006/table_0,0,145,театр,Театр им. Евгения Вахтангова (Москва) Театр им...
1,dataset/Dataset/data/1000006/table_0,1,110,пьеса,Дневник Анны ФранкФрэнсис Гудрич и Альберт Хак...
2,dataset/Dataset/data/1000006/table_0,4,101,примечание,видеофрагмент видеофрагмент телеспектакль виде...
3,dataset/Dataset/data/1000006/table_1,2,73,название,"Кулагин и партнёры Люба, дети и завод… Врачебн..."
4,dataset/Dataset/data/1000006/table_1,0,22,год,2004—2013 2006 2006 2006 2008 2007 2007 2008 2...
...,...,...,...,...,...
1441343,dataset/Dataset/data/999985/table_0,7,66,место,1 2 2 1 1 3 3 2 1
1441344,dataset/Dataset/data/999987/table_0,6,118,результат,7½ из 9 Нокаут-система 6½ из 9 8 из 11 6½ из 1...
1441346,dataset/Dataset/data/999987/table_0,0,22,год,2002 2004 2010 2012 2012 2013 2014 2016 2018
1441347,dataset/Dataset/data/999987/table_0,7,66,место,1 2 2 1 1 3 3 2 1


In [25]:
# tokenizer will take care of it
# df_joined.loc[:, "column_data"] = df_joined["column_data"].progress_apply(lambda x: str(x).lower())

### Save dataset

In [46]:
test_df.to_csv("./test/data.csv", sep="|", index=False)

In [48]:
import numpy as np

number_of_chunks = 21
for id, chunk in enumerate(np.array_split(train_df, number_of_chunks)):
    chunk.to_csv(f"./train/data_{id}.csv", sep="|", index=False)